<a href="https://colab.research.google.com/github/andreaaraldo/machine-learning-for-networks/blob/master/9x.ml_highspeed_networks/Testbed%20experimental%20emulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from multiprocessing import Queue, Process, Event
import pandas as pd
import numpy as np
import queue
import time
import logging

After importing the main libraries, we can create a shared queue which simulates the physical link between two machines.


TXGEN ---> [ shared queue ] ----> RX ----> Processing



In [0]:
shared = Queue(maxsize=1024)   # Max size of the queue
rate = .1 # Rate for the TX generation
total_number = 10000  # Max number of packets to transmit
duration = 5

In [0]:
# TX gen function
def txgen(id, l_queue, stop_event):
    count= 0
    lost = 0

    # Read the tx dataset and transform to numpy
    full_df = pd.read_csv('generator.csv')
    data = full_df.to_numpy()
    
    # Limit to iterate over the dataset
    # Feat to get rid of the labels in the csv
    limit = len(data[:,0])
    feat = len(data[0,:]) - 2
    
    while (not stop_event.is_set() ):
#    while (total_number > 0 ):

        try:
            l_queue.put_nowait( data[count%limit,:feat] )
            logging.debug("Packet added to the queue " +str(count))
        except queue.Full:
            logging.debug("Packet loss!")
            lost += 1 


        count += 1 
        logging.debug("Total packet sent " +str(count))
        stop_event.wait(timeout=.00001)#        print ("working on %s" % arg)

    logging.info("Sent: %d Lost %d ", count, lost)

In [0]:
# PROCESSING FUNCTION: You have to modify this function
# Hint: Import the traned model, perform the classification task, and then return
def processing(element):

    ##################################
    # Your processing goes her
    # after loading a model
    # use it to process the element
    #
    # y_pred = model(element)
    #
    # You should return the value of the classification task y_pred
    #  
    # Bonus: You can also compare with the original y from the csv
    ##################################
    return (1)

In [0]:
# RX Function
def rx(id, l_queue, stop_event):
    logging.debug("Starting the consumer")
    count = 0

    while (not stop_event.is_set()):

        try:
            logging.debug("Reading queue")
            pkt = l_queue.get(timeout=1)
            logging.debug("Retrieved element")

            # Processing starting... First counter
            #t0 = time.clock()
            count+=1

            #################################
            # Processing function. Here you have to put your ML approach
            # Pkt is already a numpy element, including all the features but no labels
            # The processing task is to classify the pkt
            processing(pkt)
            #################################

            #logging.debug("Elapsed time: %.6f", time.clock() - t0)
            logging.debug("Count: %d", count)
            
        except Exception as e:
            print (e)
            pass
    logging.info("Received: %d", count)


In [0]:
def main():
    # Hint: use a global model loaded from the file where you saved your training model
    
    
    # Event variables for the experiment
    producer_stop = Event()
    consumer_stop = Event()

    # Logger: set the logger to level INFO for normal usage, DEBUG for detailed info
    logging.basicConfig(level=logging.INFO)

    # Here we start the Traffic generator
    t = Process(target=txgen, args=(0, shared, producer_stop))
    t.start()

    # Here we start the receiver
    t2 = Process(target=rx, args=(0, shared, consumer_stop))
    t2.start()

    # Experiment duration
    time.sleep(duration)
    producer_stop.set()
    time.sleep(1)
    consumer_stop.set()

    t.join()
    t2.join()

if (__name__ == "__main__"):
    main()

INFO:root:Sent: 65636 Lost 0 


INFO:root:Received: 65636


In [0]:
print ("The end")

The end
